In [1]:
from dataset import WandDataset, HomeDepotDataset

In [2]:
wand = WandDataset()

In [5]:
wand.get_query()[:5]

array([[0, 'salon chair'],
       [1, 'smart coffee table'],
       [2, 'dinosaur'],
       [3, 'turquoise pillows'],
       [4, 'chair and a half recliner']], dtype=object)

In [6]:
wand.get_docs()[:5]

[Document(page_content='good , deep sleep can be quite difficult to have in this busy age . fortunately , there ’ s an antidote to such a problem : a nice , quality bed frame like the acacia kaylin . solidly constructed from acacia wood , this bed frame will stand the test of time and is fit to rest your shoulders on for years and years . its sleek , natural wood grain appearance provides a pleasant aesthetic to adorn any bedroom , acting both as a decorative piece as well as a place to give comfort after a hard day of work . our bed frame is designed to give ample under-bed space for easy cleaning and other usages , with a headboard attached to further express the craftiness . it can be used with other accessories such as a nightstand or bookcase headboard and is compatible with many types of mattresses including memory foam , spring , or hybrid ones . there ’ s nowhere better to relax than your own home , and with this bed frame that feeling of homeliness will even be more emphasized

In [ ]:
query_id = 0
product_id_list = [25434, 12088]
wand.get_score(query_id, product_id_list)

In [ ]:
# home
home = HomeDepotDataset()

In [ ]:
home.get_query().head()

In [ ]:
home.get_docs()[:5]

In [ ]:
query_id = 0
product_id_list = [100001, 101036]
home.get_score(query_id, product_id_list)

In [ ]:
from Evaluation import Evaluator

evaluator = Evaluator()

y_true = [[0, 1, 1, 1]]
y_score = [[3, 2, 1, 0]]

print(evaluator.evaluate(y_true, y_score, 10))


In [ ]:
# import modules
import numpy as np
import pandas as pd
from tabulate import tabulate

# import utilities
from BM25 import BestMatching
from VectorSearch import (
    BasicSearch,
    HyDESearch,
    HyDEWithCustomSearch,
)
from CrossEncoderReRanking import CrossEncoder

from Evaluation import Evaluator

from utilities import (
    initialize_model,
    load_docs,
    pd_multi_merge,
)

In [ ]:
# 評価
top_k = 10
llm, embeddings = initialize_model()

# vector search
bestMatching = BestMatching(load_docs())
basicSearch = BasicSearch(llm, embeddings)
hyDESearch = HyDESearch(llm, embeddings)
hyDEWithCustomSearch = HyDEWithCustomSearch(llm, embeddings)
crossEncoder = CrossEncoder(bestMatching, basicSearch)
evaluator = Evaluator()

queries = ["キャリア自律について知りたい", "コミック"]
y_trues = [np.array([[0,1,0,0,0,1]]), np.array([[0,0,1,1,1,0]])]
y_scores = []

for query, y_true in zip(queries, y_trues):
    # keyword search
    df_bm25 = bestMatching.search(query, top_k).rename(columns={'score': 'bm25'})['bm25']

    df_preprocess = pd_multi_merge([
        basicSearch.search(query, table_name='original', top_k=top_k).rename(columns={'score': 'basic_original'})['basic_original'],
        hyDESearch.search(query, table_name='original', top_k=top_k).rename(columns={'score': 'hyde_original'})['hyde_original'],
        hyDEWithCustomSearch.search(query, table_name='original', top_k=top_k).rename(columns={'score': 'hyde_custom_original'})['hyde_custom_original'],
        crossEncoder.search(query, table_name='original', top_k=top_k).rename(columns={'score': 'cross_encoder_original'})['cross_encoder_original'],
    ], left_index=True, right_index=True, how='outer')

    tables = ['synopsis', 'synopsis_ja', 'summary', 'summary_ja', 'split']
    df_tables = pd_multi_merge([
        basicSearch.search(query, table_name, top_k=top_k).rename(columns={'score': f'basic_{table_name}'})[f'basic_{table_name}']
        for table_name in tables
    ], left_index=True, right_index=True, how='outer')

    df_all = pd_multi_merge([df_bm25, df_preprocess, df_tables], left_index=True, right_index=True, how='outer')
    print(tabulate(df_all, headers='keys', tablefmt='psql'))

df_all

In [8]:
from dataset import (
    WandDataset,
    HomeDepotDataset,
)

WANDS = WandDataset()
base_docs = WANDS.get_docs()

[Document(page_content='good , deep sleep can be quite difficult to have in this busy age . fortunately , there ’ s an antidote to such a problem : a nice , quality bed frame like the acacia kaylin . solidly constructed from acacia wood , this bed frame will stand the test of time and is fit to rest your shoulders on for years and years . its sleek , natural wood grain appearance provides a pleasant aesthetic to adorn any bedroom , acting both as a decorative piece as well as a place to give comfort after a hard day of work . our bed frame is designed to give ample under-bed space for easy cleaning and other usages , with a headboard attached to further express the craftiness . it can be used with other accessories such as a nightstand or bookcase headboard and is compatible with many types of mattresses including memory foam , spring , or hybrid ones . there ’ s nowhere better to relax than your own home , and with this bed frame that feeling of homeliness will even be more emphasized

In [21]:
import numpy as np
nan_list = [doc for doc in base_docs if doc.metadata['document_id'] == 32]
nan_list

[Document(page_content='nan', metadata={'document_id': 32, 'title': "array 8.25 '' solid hardwood wall paneling in american walnut"})]

In [9]:
base_docs[:5]

[Document(page_content='good , deep sleep can be quite difficult to have in this busy age . fortunately , there ’ s an antidote to such a problem : a nice , quality bed frame like the acacia kaylin . solidly constructed from acacia wood , this bed frame will stand the test of time and is fit to rest your shoulders on for years and years . its sleek , natural wood grain appearance provides a pleasant aesthetic to adorn any bedroom , acting both as a decorative piece as well as a place to give comfort after a hard day of work . our bed frame is designed to give ample under-bed space for easy cleaning and other usages , with a headboard attached to further express the craftiness . it can be used with other accessories such as a nightstand or bookcase headboard and is compatible with many types of mattresses including memory foam , spring , or hybrid ones . there ’ s nowhere better to relax than your own home , and with this bed frame that feeling of homeliness will even be more emphasized

In [11]:
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.vectorstores import Qdrant
from langchain.vectorstores.pgvector import PGVector
from config import Settings
settings = Settings()

# モデルの初期化
def initialize_model():
    chat = AzureChatOpenAI(
        temperature=0,
        openai_api_key=settings.AZURE_OPENAI_API_KEY,
        openai_api_base=settings.AZURE_OPENAI_API_ENDPOINT,
        deployment_name=settings.AZURE_OPENAI_API_DEPLOYMENT_NAME,
        openai_api_version=settings.AZURE_OPENAI_API_VERSION,
        openai_api_type="azure",
        max_tokens=1000,
    )

    embeddings = OpenAIEmbeddings(
        openai_api_key=settings.AZURE_OPENAI_API_KEY,
        openai_api_base=settings.AZURE_OPENAI_API_ENDPOINT,
        openai_api_version=settings.AZURE_OPENAI_API_VERSION,
        openai_api_type="azure",
        model=settings.EMBEDDING_MODEL_NAME,
        chunk_size=1
    )
    return chat, embeddings

# ベクトルストアに保存
def saveVector(embeddings, docs, tabel_name):

    # Qdrant
    Qdrant.from_documents(
        documents=docs,
        embedding=embeddings,
        url=settings.QDRANT_URL,
        prefer_grpc=True,
        collection_name=tabel_name,
    )

In [13]:
# モデルの初期化
chat, embeddings = initialize_model()

# 原文のまま
saveVector(embeddings, base_docs[:5], "original")